<a href="https://colab.research.google.com/github/SREEPERAMBUDURU/projects-MachineLearning/blob/main/rfm_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from openpyxl import load_workbook

try:
    wb = load_workbook(filename="/content/online_retail_II.xlsx")
    print(wb.sheetnames)  # Print sheet names to verify the file was loaded correctly
except Exception as e:
    print("An error occurred:", e)


['Year 2009-2010', 'Year 2010-2011']


In [4]:

import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)

df = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df.head()
# Number of NaN values per column:
print(df.isnull().sum())

print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

df.dropna(inplace=True)

df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[(df['Quantity'] > 0)]

df.describe([0.01,0.25,0.50,0.75,0.99]).T

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df.describe([0.01,0.25,0.50,0.75,0.99]).T

df["TotalPrice"] = df["Quantity"] * df["Price"]

today_date = dt.datetime(2011, 12, 11)
print(f" Maximum invoice date: {df.InvoiceDate.max()} \n Today date: {today_date}")

rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days,
                                     "Invoice": lambda num: num.nunique(),
                                      "TotalPrice": lambda price: price.sum()}) #total price per customer

rfm.columns = ['Recency', 'Frequency', "Monetary"]
rfm.reset_index(inplace=True)
rfm.head()

rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])

rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))

seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'}

rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

Customer_ID = 12431.0
rfm[rfm["Customer ID"] == Customer_ID]["Segment"]

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64
                 0.00      0.05      0.50      0.95     0.99     1.00
Quantity    -80995.00      1.00      3.00     29.00    100.0  80995.0
Price       -11062.06      0.42      2.08      9.95     18.0  38970.0
Customer ID  12346.00  12626.00  15152.00  17905.00  18212.0  18287.0


<ipython-input-4-e5113915273d>:10: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)


 Maximum invoice date: 2011-12-09 12:50:00 
 Today date: 2011-12-11 00:00:00


69    Loyal_Customers
Name: Segment, dtype: object

In [ ]:
print(df.isnull().sum())
print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
print(df.dtypes)

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64
                0.00      0.05      0.50     0.95      0.99       1.00
Quantity         1.0      1.00      6.00     36.0    120.00    298.500
Price            0.0      0.42      1.95      8.5     14.95     37.060
Customer ID  12346.0  12627.00  15159.00  17912.0  18211.00  18287.000
TotalPrice       0.0      1.25     11.80     67.5    183.60   3268.575
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object


<ipython-input-26-9dba1edd41fe>:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
